# Análise de Dados Aplicada a Recursos Humanos

Uma empresa de consultoria busca Cientistas de Dados com qualificações em treinamentos ministrados pela própria empresa. O objetivo da empresa é identificar candidatos com perfil adequado e realmente interessados em trabalhar para ela ao concluírem os treinamentos. O objetivo da análise é realizar um extenso trabalho de análise nos dados, compreender como estão organizados, realizar limpeza e transformação dos dados conforme necessário, detectar eventuais padrões e diagnósticos e ao final emitir um parecer com um resumo sobre os dados.

## Instalando e Importando Pacotes.

In [ ]:
# Instala o pacote watermark. Utilizado para obter informações sobre as versões de outros pacotes utilizados neste Jupyter Notebook.
# https://pypi.org/project/watermark/
%pip install -q watermark

In [ ]:
# Carrega o pacote watermark como um comando mágico no Jupyter Notebook.
%load_ext watermark

In [ ]:
# Instala o pacote missingno. Utilizado para criação de mapa de valores ausentes.
# https://pypi.org/project/missingno/
%pip install -q missingno

In [ ]:
# Instala o pacote plotly. Utilizado para criação de gráficos.
# https://pypi.org/project/plotly/
%pip install -q plotly

In [ ]:
# Importa pacotes necessários para esse projeto.

# Manipulação de dados
import pandas as pd
import numpy as np

# Visualização
import missingno
import matplotlib.pyplot as plt
import seaborn as sns

# Estatística
from scipy.stats import normaltest

In [ ]:
# Versão do Python e de todos os pacotes importados neste projeto.
%watermark -a "Luciana Sampaio" -gu "luasampaio" --python --iversions --watermark

## Análise Exploratória de Dados

### Carregando o conjunto de dados

In [ ]:
#Carrega o dataset como um Dataframe do pandas.
df = pd.read_csv('dataset/aug_train.csv')

### Informações Gerais

In [ ]:
# Quantidade de linhas e colunas do conjunto de dados.
rows, columns = df.shape
print(f"Linhas: {rows}\nColunas: {columns}")

In [ ]:
# Colunas do conjunto de dados.
df.columns

In [ ]:
# Amostra dos dados. Primeiras cinco linhas do conjunto de dados.
df.head()

In [ ]:
# Visualização dos tipos de dados.
df.info()

### Visualizando as Variáveis Categóricas

In [ ]:
# Descreve os dados categóricos.
df.describe(include = object)

In [ ]:
# Plot

# Tamanho da figura.
plt.figure(figsize = (18,30))

# Lista de colunas categóricas.
column_list = list(df.columns.values)[3:12]

# Contador.
contador = 1

for column in column_list:

    plt.subplot(5, 2, contador) # Tamanho individual de cada sub-gráfico.
    ax = sns.countplot(data = df.fillna('NaN'), x = column) #Conta as ocorrências, substituindo NaN pela string "NaN" através de fillna.
    plt.title(column, fontsize = 15) #Titulo do Sub-Gráfico.

    # Valor quantitativo de cada barra.
    for p in ax.patches:
        ax.annotate(f'{p.get_height()}', (p.get_x() + 0.4, p.get_height()), ha = 'center', color = 'black', size = 12)
    
    # A partir do sétimo gráfico rotacionar a legenda de cada coluna em 45°.
    if contador >= 7:
        plt.xticks(rotation = 45)

    contador += 1 # Incrementa o contador em 1.

# Layout
plt.tight_layout(h_pad = 2)

### Verificando a Distribuição das Variáveis Numéricas

In [ ]:
# Descrevendo os dados numéricos mas retirando os campos 'enrollee_id' e 'target', pois apesar de serem dados numéricos são, respectivamente, variáveis de identificador e categoria.
df.describe().drop(columns = ['enrollee_id', 'target'])

* Em **city_development_index** (CDI), os valores médios são 0,828, mediana 0,903 e desvio padrão 0,123. Isso significa que a maioria dos candidatos é de cidades bem desenvolvidas. O Índice que mede desenvolvimento dos municípios são divididos em quatro categorias: baixo desenvolvimento (de 0 a 0,4), desenvolvimento regular (0,4 a 0,5), desenvolvimento moderado (de 0,6 a 0,8) e alto desenvolvimento (0,8 a 1).


* Em **training_hours**, os valores médios são 65,367, mediana 47 e máximo 336. Isso significa que há mais candidatos com poucas horas de treinamento, mas alguns candidatos levam muito tempo para fazer o treinamento.

In [ ]:
# Plot

# Tamanho da figura.
plt.figure(figsize = (17,12))

# Sub-Gráficos de Histogramas.
plt.subplot(221) #Posição.
sns.histplot(data = df, x = 'city_development_index', kde = True, color = "green") #Histograma com Curva de Densidade (kde).
plt.title('Histograma do CDI', fontsize = 20) #Título

plt.subplot(222) #Posição.
sns.histplot(data = df, x = 'training_hours', kde = True, color = "magenta") #Histograma com Curva de Densidade (kde)
plt.title('Histograma das Horas de Treinamento', fontsize = 20) #Título.

# Sub-Gráficos de Boxplots.
plt.subplot(223) #Posição.
sns.boxplot(data = df, x = 'city_development_index', color = "green", orient="h") #Boxplot com orientação Horizontal.

plt.subplot(224) #Posição.
sns.boxplot(data = df, x = 'training_hours', color = "magenta", orient="h") #Boxplot com orientação Horizontal.

plt.show()

* Pela curva de densidade do histograma de **city_development_index** (CDI) é possível identificar que a variável tem uma **distribuição de frequência bimodal**, ou seja, com duas modas. Ainda é possível identificar a presença de outliers mostrado pelo Boxplot.

* Pela curva de densidade do histograma de **training_hours** é possível identificar que a variável tem uma **distribuição de frequência assimetrica positiva**, ou à direita. Ainda é possível identificar a presença de muitos outliers mostrado pelo Boxplot.


### Teste de Normalidade da Distribuição

Mesmo interpretando os gráficos anteriores é de suma importância testar a normalizade das variáveis.

In [ ]:
# Lista com as variáveis numéricas
numerical_feature = ['city_development_index', 'training_hours']

for feature in numerical_feature:
    # Calcula a normalidade através do 'teste normal'
    statistic, pvalue = normaltest(df[feature])

    # Verificando o valor-p
    if pvalue > 0.05:
        print(feature, ': Distribuição Normal')
    else:
        print(feature, ': Distribuição Não Normal')

* Como as variáveis **city_development_index** e **training_hours** não seguem a distribuição normal, usaremos o método **não paramétrico** para a análise numérica.

* Dados de **training_hours** estão localizados principalmente no lado esquerdo do histograma. É um comportamento esperado porque é normal as pessoas se animarem no início do treinamento, mas nem todas o concluem.

## Correlação de Dados e Seleção de Recursos

### Conversão de Variáveis

O interpretador da linguagem Python classifica as variáveis **experience** e **last_new_job** como object, devido a presença de caracteres textuais em seus respectivos dados. Todavia, dependendo da natureza do problema a ser resolvido e do comportamento de cada variável, elas podem ser abordadas como variáveis categóricas ou númericas. Como é necessário selecionar profissionais para uma possível contratação, informações referentes a experiência total de cada candidato (em anos) e a diferença de tempo entre e o último emprego e o atual (em anos) podem ter correlação númerica com a variável **target**.

In [ ]:
# Cria uma cópia do Dataframe original.
df_numerical = df.copy() #Dataframe temporário

In [ ]:
# Ocorrência de cada valor textual da variável experience.
df_numerical["experience"].value_counts()

In [ ]:
# Converte a variável experience para numérica.
df_numerical["experience"] = np.where(df_numerical["experience"] == "<1", 1, df_numerical["experience"]) #Substituindo <1 por 1.
df_numerical["experience"] = np.where(df_numerical["experience"] == ">20", 21, df_numerical["experience"]) #Substituindo >20 por 21.
df_numerical["experience"] = df_numerical["experience"].astype(float) #Conversão de tipos.

In [ ]:
# Ocorrência de cada valor numérico da variável experience.
df_numerical["experience"].value_counts()

In [ ]:
# Ocorrência de cada valor textal da variável last_new_job.
df_numerical["last_new_job"].value_counts()

In [ ]:
# Converte a variável last new job para numérica.
df_numerical["last_new_job"] = np.where(df_numerical["last_new_job"] == "never", 0, df_numerical["last_new_job"]) #Substituindo 'never' por 0.
df_numerical["last_new_job"] = np.where(df_numerical["last_new_job"] == ">4", 5, df_numerical["last_new_job"]) #Substituindo >4 por 5.
df_numerical["last_new_job"] = df_numerical["last_new_job"].astype(float) #Conversão de tipos.

In [ ]:
# Ocorrência de cada valor numérico da variável last_new_job.
df_numerical["last_new_job"].value_counts()

In [ ]:
# Visualização dos tipos de dados.
df_numerical.info()

In [ ]:
df_numerical.head()

In [ ]:
df_numerical.dtypes


Algumas interpretações sobre os resultados obtidos:

- Embora seja uma correlação fraca, as variáveis **experience** e **city_development_index** tem correlação positivas (0.30) entre si. Possivelmente, profissionais residentes em cidades com alto índice de desenvolvimento tem maiores experiências.

- As variáveis **experience** e **last_new_job** tem correlação média positivas (0.473) entre si. Possivelmente, profissinais que ficam pouco tempo sem trabalhar são mais experientes.

- As variáveis **experience** e **trainng_hours** não tem correlação entre si, pois o valor é muito próximo a zero (0.0036).

- Embora seja uma correlação fraca, as variáveis **experience** e **target** tem correlação negativas (-0.183) entre si. Possivelmente, profissionais com maiores experiências não estão buscando por um novo emprego na empresa de consultoria.

- As variáveis **last_new_job** e **training_hours** tem correlaões próximas de zero com a variável alvo e, portanto, podem ser descartadas.

### Weight of Evidence (WOE) e Information Value (IV)


Weight of Evidence (WOE) e Information Value (IV) são métricas usadas para avaliar a importância de uma variável para prever uma variável alvo. Eles são comumente usados com variáveis categóricas, mas também podem ser aplicados a variáveis numéricas. O WOE fornece o poder preditivo de cada classe de uma variável, enquanto o IV fornece um valor único que representa o poder preditivo total da variável. Estas métricas ajudam na seleção de recursos importantes para prever a variável alvo.

A fórmula para calcular o peso da evidência para qualquer característica é dada por:
- WOE = ln (percentual de eventos / percentual de não eventos)

A fórmula para calcular o valor da informação é dada por:
- IV = &sum; (WOE * (percentual de eventos - percentual de não eventos)

<table border = "1">
    <tr>
        <td>< 0.02</td>
        <td>Não deve ser usado para previsão</td>
    </tr>
    <tr>
        <td>0.02 - 0.1</td>
        <td>Preditor fraco</td>
    </tr>
    <tr>
        <td>0.1 - 0.3</td>
        <td>Preditor médio</td>
    </tr>
    <tr>
        <td>0.3 - 0.5</td>
        <td>Preditor forte</td>
    </tr>
    <tr>
        <td>> 0.5</td>
        <td>Bom demais para ser verdade</td>
    </tr>
</table>

Nesta etapa da análise, as seguintes variáveis serão excluídas: **enrollee_id**, pois é uma variável de identificação única; **city_development_index**, **training_hours**, **experience** e **last_new_job**, pois são variáveis numéricas; **city**, pois não há informações relevantes sobre cidades específicas; e **company_size**, pois o tamanho da empresa que o candidato trabalha atualmente não é relevante para a análise. Além disso, a variável **target** também será excluída por ser a variável alvo.

In [ ]:
#Indica quais colunas serão removidads.
drop_columns = ['enrollee_id', #Variavável de identificação.
                'city_development_index', #Variável numérica.
                'training_hours', #Variável numérica.
                'experience', #Variável numérica.
                'last_new_job', #Variável numérica.
                'city', #Variável irrelevante.
                'company_size', #Variável irrelevante.
                'target' #Variável alvo.
              ]

# Remove algumas colunas do Dataframe deixando apenas variáveis categóricas para essa análise.
categorical_columns = df.drop(columns = drop_columns).columns

# Lista para armazenar o IV.
iv = []

#Loop.
for column in categorical_columns:
    #Calcula a tabela cruzada de Target com todas as classes de todas as variaváveis que não foram retiradas.
    df_woe_iv = (pd.crosstab(df[column], df['target'], normalize = 'columns')
                .assign(woe = lambda dfx: np.log(dfx[1] / dfx[0])) #WOE.
                .assign(iv = lambda dfx: np.sum(dfx['woe'] * (dfx[1] - dfx[0])))) #IV.
    #Armazenando os valores de IV.
    iv.append(df_woe_iv['iv'][0])

    print(df_woe_iv,'\n------------------------------------------------------------')

In [ ]:
# Dataframe do Information Value
df_iv = pd.DataFrame({'Features':categorical_columns,'iv':iv}).set_index('Features').sort_values(by = 'iv')

# Plot do Information Value
plt.figure(figsize = (10,12)) #Tamanho do gráfico.
df_iv.plot(kind = 'barh', title = 'Information Value das Variáveis Categóricas', colormap = "Accent")

for index, value in enumerate(list(round(df_iv["iv"],3))):
    plt.text((value), index, str(value))

plt.legend(loc = "lower right")
plt.show()

- No gráfico acima, podemos ver a ordem dos recursos com base em seu poder preditivo em relação ao alvo.

- De acordo com o valor de Information Value (IV), **enrolled_university** é considerado um preditor de médio, enquanto que **relevant_experience** e **education_level** são considerados preditores fracos. As outras variáveis poderiam ser desconsideradas para a previsão, pois apresentam correlação próxima a zero com a variável **target**, ou ainda, não são necessárias para determinar se um(a) candidato(a) deve ser contratado(a). Entretanto, neste momento da análise apenas a variável **gender** será descartada. É possível que outras variáveis nessas mesmas condições também sejam descartadas de acordo com o resultado de análises futuras.

## Identificando Valores Ausentes, Duplicados e Desbalanceados

### Identificando Valores Ausentes

In [ ]:
# Valores ausentes (NaN) por coluna.
null_df = df.isna().sum().reset_index()

# Tamanho do gráfico.
ax = plt.figure(figsize = (10,5))

# Barplot.
ax = sns.barplot(x = null_df['index'], y = null_df[0], palette = 'husl')

#Formatação do Gráfico.
plt.xlabel('Atributos', fontsize = 12)
plt.ylabel('Contagem de Valores Ausentes', fontsize = 12)
plt.xticks(rotation = 45)
plt.title("Plot de Valores Ausentes", fontsize = 15)

for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.4, (p.get_height())), ha = 'center', color = 'black', size = 11)

plt.show()

In [ ]:
# Mapa de Valores Ausentes.

# Dataframe.
df_nan = pd.DataFrame(df.isna().sum())

# Plot.
if df.isna().any(axis = None): #A função any() retorna True se qualquer item em um objeto iterável for verdadeiro, caso contrário, retorna False. Se o objeto iterável estiver vazio, a função any() retornará False.
    missingno.matrix(df[df_nan[df_nan[0]>0].index])
    plt.show()

Os valores ausentes de cada coluna não seguem um padrão.

### Identificando Valores Duplicados

In [ ]:
# Verifica valores duplicados pela coluna de id.
df['enrollee_id'].duplicated().sum()

Não há linhas com identificadores duplicados

### Identificando Dados Desbalanceados

In [ ]:
# Tamanho da figura.
plt.figure(figsize = (17,(100)/20))

#Pieplot.
plt.subplot(121) #Posição do Sub-Gráfico.
plt.pie(round(df['target'].value_counts() / len(df) * 100, 2), labels = list(df['target'].value_counts().index), autopct = "%.2f%%", explode = (0,0.1))
plt.axis("equal") #Espaço entre os Sub-Gráficos.
plt.title("Target Imbalance Ratio", size = 10) #Título.

#Barplot.
plt.subplot(122) #Posição do Sub-Gráfico.
ax = sns.countplot(data = df, x = 'target')
plt.title("Barplot Target Label", fontsize = 10) #Título.

for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.4, p.get_height()), ha = 'center', va = 'top', color = 'white', size = 12)

Os dados da variável **target** estão desbalanceados. Para o tipo de análise realizada nesse momento uma correção não é necessária. Em outras analises, como a preditiva, seria imprescindível corrigí-los.

## Limpeza e Processamento dos Dados

### Tratando os Valores Ausentes

Através das análises realizadas anteriormente, algumas variáveis foram identificadas como não sendo relevantes para o problema de negócio em questão, e, portanto, não serão incluídas nas futuras análises ou modelos. Recapitulando:

- Os atributos **city** e **company_size** foram considerados irrelevantes pelo contexto do problema de negócio.
- O atributo categórico **gender** não será utilizado pois tem Information Value (IV) próximo a zero com a variável **target**.
- Os atributos númericos **last_new_job** e **training_hours** não serão utilizados pois têm correlações de Spearman próximas de zero com a variável **target**. 

In [ ]:
df.columns

In [ ]:
#Seleção de colunas relevantes para a análise.
columns_use = ['city_development_index', 
               'experience', 
               'enrolled_university', 
               'relevent_experience', 
               'education_level',
               'company_type',
               'major_discipline',
               'target']

In [ ]:
#Novo conjunto de dados sem as variáveis indesejadas.
new_df = df[columns_use]

In [ ]:
# Amostra dos dados. Primeiras cinco linhas do novo conjunto de dados.
new_df.head()

In [ ]:
# Valores ausentes (NaN) por coluna do novo conjunto de dados.
null_df = new_df.isna().sum().reset_index()

# Tamanho do gráfico.
ax = plt.figure(figsize = (15,5))

# Barplot.
ax = sns.barplot(x = null_df['index'], y = null_df[0], palette = 'husl')

#Formatação do Gráfico.
plt.xlabel('Atributos', fontsize = 12)
plt.ylabel('Contagem de Valores Ausentes', fontsize = 12)
plt.xticks(rotation = 45)
plt.title("Plot de Valores Ausentes", fontsize = 15)

for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.4, (p.get_height())), ha = 'center', color = 'black', size = 11)

plt.show()

In [ ]:
# Percentual de valores ausentes
percent_missing = new_df.isnull().sum() / len(new_df) * 100
percent_missing

#### Variável **major_discipline**

In [ ]:
# Valores ausentes da Variável major_discipline.
# Major_discipline quando se tem maior valor acadêmico em relação aos demais. 
sns.countplot(data = new_df.fillna('NaN'), x = 'major_discipline', alpha = 0.7, edgecolor = 'black')
#Formatação
plt.xticks(rotation = 45)
bound = ax.get_xbound()
ax = plt.gca()
for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.4, p.get_height()), ha = 'center', color = 'black', size = 10)

plt.title("Valores Ausentes da Variável major_discipline\n", fontsize = 15)
plt.show()

##### Estratégia 01

Relação de valores ausentes entre major_discipline x education_level

In [ ]:
print('Total de Valores Ausentes na Variável major_discipline:', new_df['major_discipline'].isna().sum())
print('Proporção de Valores Ausentes na Variável education_level:')
new_df[new_df['major_discipline'].isna()]['education_level'].value_counts(dropna = False)

Analisando os valores ausentes da variável **major_discipline**, é possível observar a proporção de acordo com a variável **education_level**. A fim de preencher os valores faltantes da variável **major_discipline**, optou-se por utilizar **Non Degree** quando houverem registros de High School ou Primary School na variável **education_level**. Isso significa que o candidato não tem formação em nível de especialização.

In [ ]:
#Selecionando os índices que serão modificados.
nan_index = new_df[new_df['major_discipline'].isna() & new_df['education_level'].isin(['High School', 'Primary School', np.nan])].index

In [ ]:
#Tamanho do index.
len(nan_index)

In [ ]:
# Imputação do valor ausente.
new_df.loc[nan_index, 'major_discipline'] = 'Non Degree'

##### Estratégia 02

In [ ]:
print('Total de Valores Ausentes na Variável major_discipline:', new_df['major_discipline'].isna().sum())
new_df['major_discipline'].value_counts(dropna = False)

Para os valores ausentes restantes preenchemos com 'Other'

In [ ]:
new_df.loc[new_df['major_discipline'].isna(), 'major_discipline']  = 'Other'

In [ ]:
# Valores Corrigidos da Variável major_discipline
sns.countplot(data = new_df.fillna('NaN'), x = 'major_discipline', alpha = 0.7, edgecolor = 'black')
#Formatação
plt.xticks(rotation = 45)
bound = ax.get_xbound()
ax = plt.gca()
for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.4, p.get_height()), ha = 'center', color = 'black', size = 10)

plt.title("Valores Corrigidos da Variável major_discipline\n", fontsize = 15)
plt.show()

#### Variável **enrolled_university**

In [ ]:
# Valores ausentes específicos da Variável enrolled_university.
sns.countplot(data = new_df.fillna('NaN'), x = 'enrolled_university', alpha = 0.7, edgecolor = 'black')
# Formatação.
sns.despine()
plt.xticks()
bound = ax.get_xbound()
ax = plt.gca()

for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.4, p.get_height()), ha = 'center', color = 'black', size = 10)

plt.title("Valores Ausentes da Variável enrolled_university\n", fontsize = 15)
plt.show()

##### Estratégia 01

Relação de valores ausentes entre major_discipline x education_level

In [ ]:
print('Total de Valores Ausentes na Variável major_discipline:', new_df['enrolled_university'].isna().sum())
print('Proporção de Valores Ausentes na Variável education_level:')
new_df[new_df['enrolled_university'].isna()]['education_level'].value_counts(dropna = False)

 Analisando os valores ausentes da variável **enrolled_university**, é possível observar a proporção de acordo com a variável **education_level**. A fim de preencher os valores faltantes da variável **enrolled_university**, optou-se por utilizar **Primary Grad** quando houverem registros de Primary School na variável **education_level**. Isso significa que o candidato não está apto a ingressar na universidade.

In [ ]:
# Selecionando os índices que serão modificados
nan_index = new_df[new_df['enrolled_university'].isna() & new_df['education_level'].isin(['Primary School'])].index

In [ ]:
len(nan_index)

In [ ]:
# Imputação de valores ausentes
new_df.loc[nan_index, 'enrolled_university'] = 'Primary Grad'

##### Estratégia 02

In [ ]:
print('Total de Valores Ausentes na Variável major_discipline:', new_df['enrolled_university'].isna().sum())
new_df['enrolled_university'].value_counts(dropna = False)

Para os valores ausentes restantes preenchemos com 'Other'

In [ ]:
new_df.loc[new_df['enrolled_university'].isna(), 'enrolled_university']  = 'Other'

In [ ]:
# Valores Corrigidos da Variável major_discipline
sns.countplot(data = new_df.fillna('NaN'), x = 'enrolled_university', alpha = 0.7, edgecolor = 'black')
#Formatação
plt.xticks(rotation = 45)
bound = ax.get_xbound()
ax = plt.gca()
for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.4, p.get_height()), ha = 'center', color = 'black', size = 10)

plt.title("Valores Corrigidos da Variável major_discipline\n", fontsize = 15)
plt.show()

#### Variável **company_type**

In [ ]:
# Valores ausentes específicos da Variável major_discipline
sns.countplot(data = new_df.fillna('NaN'), x = 'company_type', alpha = 0.7, edgecolor = 'black')
#Formatação
plt.xticks(rotation = 45)
bound = ax.get_xbound()
ax = plt.gca()
for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.4, p.get_height()), ha = 'center', color = 'black', size = 10)

plt.title("Valores Ausentes da Variável company_type\n", fontsize = 15)
plt.show()

##### Estratégia

Para os valores ausentes preenchemos com 'Other'

In [ ]:
new_df.loc[new_df['company_type'].isna(), 'company_type']  = 'Other'

In [ ]:
# Valores Corrigidos da Variável major_discipline
sns.countplot(data = new_df.fillna('NaN'), x = 'company_type', alpha = 0.7, edgecolor = 'black')
#Formatação
plt.xticks(rotation = 45)
bound = ax.get_xbound()
ax = plt.gca()
for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.4, p.get_height()), ha = 'center', color = 'black', size = 10)

plt.title("Valores Corrigidos da Variável company_type\n", fontsize = 15)
plt.show()

#### Variável **education_level**

In [ ]:
# Valores ausentes específicos da Variável major_discipline
sns.countplot(data = new_df.fillna('NaN'), x = 'education_level', alpha = 0.7, edgecolor = 'black')
#Formatação
plt.xticks(rotation = 45)
bound = ax.get_xbound()
ax = plt.gca()
for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.4, p.get_height()), ha = 'center', color = 'black', size = 10)

plt.title("Valores Ausentes da Variável company_type\n", fontsize = 15)
plt.show()

##### Estratégia

Para os valores ausentes preenchemos com 'Other'

In [ ]:
new_df.loc[new_df['education_level'].isna(), 'education_level']  = 'Other'

In [ ]:
# Valores Corrigidos da Variável major_discipline
sns.countplot(data = new_df.fillna('NaN'), x = 'education_level', alpha = 0.7, edgecolor = 'black')
#Formatação
plt.xticks(rotation = 45)
bound = ax.get_xbound()
ax = plt.gca()
for p in ax.patches:
    ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.4, p.get_height()), ha = 'center', color = 'black', size = 10)

plt.title("Valores Corrigidos da Variável company_type\n", fontsize = 15)
plt.show()

#### Variável **experience**

##### Estratégia

A quantidade de valores ausentes da variável experience corresponde a aproximadamente 0.33% do Dataframe. Logo, é mais viável remover os valores ausentes nesse ponto da análise.

In [ ]:
# Quantidade de linhas e colunas do conjunto de dados.
rows, columns = new_df.shape
print(f"Linhas: {rows}\nColunas: {columns}")

In [ ]:
new_df = new_df.dropna() #Deve-se usar com cautela.

In [ ]:
rows, columns = new_df.shape
print(f"Linhas: {rows}\nColunas: {columns}")

In [ ]:
# Percentual de valores ausentes
percent_missing = new_df.isnull().sum() * 100 / len(new_df)
percent_missing

### Ajustes Finais

Padronizando o campo **enrolled_university** para que a primeira letra de 'no_enrollment' seja maiúscula.

In [ ]:
new_df.head()

In [ ]:
# Padronizando os dados
new_df['enrolled_university'] = new_df['enrolled_university'].replace('no_enrollment', 'No enrollment')

In [ ]:
new_df.head()

Dividindo em x, que é usado para armazenar as features, e y, que é usado para armazenar o target. Essas variáveis são frequentemente usadas em modelos de aprendizado de máquina para treinar o modelo e fazer previsões. 

In [ ]:
x = new_df.drop(columns = ['target'])
y = new_df['target']

## Relatório Final

Com base em nossa análise observamos que as variáveis mais relevantes para identificar um bom candidato são:

- Índice de Desenvolvimento da cidade onde mora o candidato.
- Tempo de experiência profissional.
- Se está ou não matriculado em um curso universitário.
- Se tem ou não experiência relevante.
- O nível educacional.
- O tipo de empresa que o candidato trabalhou ou trabalha atualmente.
- A especialização na graduação (quando for o caso).

Não são relevantes para a análise:

- O ID do candidato.
- O código da cidade do candidato.
- O gênero.
- A última vez que o candidato esteve empregado.
- O tamanho da empresa (quando for o caso).
- Total de horas de treinamento.


**Recomendações do Analista de Dados**:

- O RH pode desenvolver um método de coleta de dados para obter outros recursos a fim de melhorar a qualidade dos dados e tornar o trabalho de análise mais preciso.

- O RH pode procurar candidatos que vêm de cidades com índice de desenvolvimento urbano mais baixo, sem experiência relevante, nível de educação superior e menor experiência de trabalho para ter maior chance de encontrar candidatos que estão procurando um emprego.

- O RH pode tornar o treinamento mais compacto porque muitas pessoas não precisam de muito tempo para concluir o treinamento.